In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('minerva_university_responses.csv')
df.dropna(subset = ['Poll Responses Response'], inplace=True)

# df.sort_values(by='Assessment reports Hashtag', axis=1, inplace=True)

# # set the index to be this and don't drop
# df.set_index(keys=['Assessment reports Hashta'], drop=False,inplace=True)

# # get a list of names
# hcs_los =df['Assessment reports Hashta'].unique().tolist()

# # now we can perform a lookup on a 'view' of the dataframe
# networks = df.loc[df.hcs_los=='networks']

grouped = df.groupby(['Assessment reports Hashtag'])
l_grouped = list(grouped)


lst = [v for k, v in df.groupby('Assessment reports Hashtag')]


In [23]:
type(lst[0]['Assessment reports Hashtag'])
lst[0]['Assessment reports Hashtag']

129170    # TheoreticalLevelsofAnalysis
129171    # TheoreticalLevelsofAnalysis
129172    # TheoreticalLevelsofAnalysis
129173    # TheoreticalLevelsofAnalysis
129174    # TheoreticalLevelsofAnalysis
                      ...              
177305    # TheoreticalLevelsofAnalysis
177306    # TheoreticalLevelsofAnalysis
177307    # TheoreticalLevelsofAnalysis
177308    # TheoreticalLevelsofAnalysis
177309    # TheoreticalLevelsofAnalysis
Name: Assessment reports Hashtag, Length: 78, dtype: object

In [6]:
grouped.head()

,id,Polls ID,Assessment reports Student ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score
0,1,12522.0,41.0,The strengths of Plato's approach is his const...,#objectivemorality,2.0
1,2,12522.0,42.0,Having a difficulty with his Divided Line. Th...,#deductivearg,2.0
2,3,12522.0,44.0,"How can one know if he's in the cave or not, i...",#objectivemorality,3.0
3,4,12522.0,237.0,S: the hierarchy of the chart we filled out at...,#objectivemorality,3.0
4,5,12522.0,239.0,It is difficult to grasp how to apply Plato's ...,#objectivemorality,3.0
...,...,...,...,...,...,...
197966,197967,322258.0,581.0,No idea. An algorithm is a decision procedure ...,#algorithmicstrategies,1.0
197967,197968,322258.0,617.0,Programs need algorithms to work. Programs are...,#algorithmicstrategies,3.0
197968,197969,322258.0,650.0,"A program is the ""full-fledged"" platform which...",#algorithmicstrategies,3.0
197969,197970,322258.0,651.0,A program implements algorithms. Algorithms ar...,#algorithmicstrategies,3.0


In [25]:
df_dict = {g: d for g, d in df.groupby('Assessment reports Hashtag')}
print(df_dict.keys())


dict_keys(['# TheoreticalLevelsofAnalysis', '#2statesystems', '#ALFtools', '#AetiologyImpactIntervention', '#AnimalModels', '#BuckinghamPiscaling', '#CognitionAndEmotionCollaboration', '#CognitionAndEmotionIntegration', '#CognitionandEmotionAnalysis', '#CognitiveControlofEmotion', '#CognitiveDevelopmentAndDecline', '#ComputationalNeurosci', '#ComputationalTools', '#ConvergingApproaches', '#CritiqueExpDesign', '#DA', '#Diffapplication', '#Difftheory', '#EMfieldsforcemotion', '#ERM', '#EfficiencyVsRisk', '#EmotionInfluenceOnCognition', '#EmotionsandCognition', '#Entropy', '#Heat', '#HumanCognition', '#IT', '#IdealGas', '#InfoVal', '#Intapplication', '#Inttheory', '#IsingModel', '#LPimplementation', '#LPinterpretation', '#LinearModeling', '#LinearSystems', '#LinearTransformations', '#MathMethods', '#MentalThriving', '#MfgVsService', '#NeuralSubstrate', '#NeuroscienceEthicsAndSociety', '#OpsAnalytics', '#OpsFirmPerf', '#Participation', '#PhaseTransitions', '#Preparedness', '#Rimplementatio

In [29]:
df_dict['#networks']

,id,Polls ID,Assessment reports Student ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score
3207,3208,31079.0,247.0,#networks 2 for the first reading (Winners Don...,#networks,4.0
3212,3213,31079.0,327.0,4 in #networks. The Ohtsuki paper introduces t...,#networks,3.0
3400,3401,32921.0,619.0,I think both #utility and #motivation are key ...,#networks,3.0
3405,3406,32921.0,629.0,When Zili was addressing the point of what are...,#networks,3.0
3408,3409,32921.0,701.0,After my point about investors with desire for...,#networks,3.0
...,...,...,...,...,...,...
197569,197570,322031.0,228.0,The reductionist approach operates under the a...,#networks,4.0
197572,197573,322031.0,267.0,The reductionist approach works from a linear ...,#networks,4.0
197581,197582,322031.0,328.0,The reductionist one assumes that one can unde...,#networks,4.0
197617,197618,322040.0,333.0,I think #decisiontree and #networks is incredi...,#networks,3.0


In [30]:
df = pd.read_csv('minerva_university_responses.csv')
df.dropna(subset = ['Poll Responses Response'], inplace=True)
df_dict = {g: d for g, d in df.groupby('Assessment reports Hashtag')}
networks = df_dict['#networks']
networks['text_clean'] = networks['Poll Responses Response'].apply(lambda x: gensim.utils.simple_preprocess(x))
# Encoding the label column
networks['Assessment reports Score']=networks['Assessment reports Score'].map({0:'N/A',1:'contains_nothing',2:'contains_some',3:'contains_enough',4:'contains_more_than_enough',5:'woof'})


In [31]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(networks['text_clean'], networks['Assessment reports Score'] , test_size=0.2)

In [32]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [34]:
#w2v_model.wv.index_to_key

In [38]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('node')

[('agent', 0.9987587928771973),
 ('another', 0.9987388849258423),
 ('reachable', 0.998473584651947),
 ('student', 0.9983173608779907),
 ('every', 0.9980826377868652),
 ('person', 0.9980599284172058),
 ('hub', 0.9979709386825562),
 ('one', 0.9978668093681335),
 ('central', 0.9978483319282532),
 ('if', 0.9978447556495667)]